In [9]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import fsolve

def U(phi, k):
    # first integral: 0 → imag(phi)
    f1 = lambda y: 1.0 / np.sqrt(1 - k**2 * np.sin(1j * y)**2)
    I1, _ = quad(lambda y: np.real(f1(y)), 0, np.imag(phi)) \
         + 1j * quad(lambda y: np.imag(f1(y)), 0, np.imag(phi))

    # second integral: 0 → real(phi)
    f2 = lambda y: 1.0 / np.sqrt(1 - k**2 * np.sin(1j*np.imag(phi) + y)**2)
    I2, _ = quad(lambda y: np.real(f2(y)), 0, np.real(phi)) \
         + 1j * quad(lambda y: np.imag(f2(y)), 0, np.real(phi))

    return 1j * I1 + I2
def solve_phi(u, k):
    def F(vars):
        phi = vars[0] + 1j*vars[1]
        val = np.sin(U(phi, k)) - np.sin(u)
        return [np.real(val), np.imag(val)]

    sol = fsolve(F, x0=[0.0, 0.0])
    return sol[0] + 1j*sol[1]

def L(k):
    f = lambda y: 1.0 / np.sqrt(1 - k**2 * np.sin(y)**2)
    val, _ = quad(f, 0, np.pi/2)
    return val

def V(k):
    m = k**2
    kp = np.sqrt(1 - m)
    kprime = (1 - kp) / (1 + kp)
    return 2 * L(kprime)

def compute_moduli(delta1, delta2, w_p, w_s):
    # ε (epsilon)
    eps = np.sqrt((2 * delta1 - delta1**2) / (1 - 2*delta1 + delta1**2))
    # k1 and k1'
    k1 = eps / np.sqrt(delta2**2 - 1)
    k1p = np.sqrt(1 - k1**2)
    # k and k'
    k = w_p / w_s
    kp = np.sqrt(1 - k**2)
    return eps, k1, k1p, k, kp

def check_moduli(k, kp, k1, k1p):
    ok_k  = (k  < 1 - 1e-9)
    ok_kp = (kp < 1 - 1e-9)
    ok_k1  = (k1  < 1 - 1e-9)
    ok_k1p = (k1p < 1 - 1e-9)

    return ok_k, ok_kp, ok_k1, ok_k1p

def K(k):
    return ellipk(k**2)

def filter_order(k, kp, k1, k1p):

    numerator   = K(k)  * K(k1p)
    denominator = K(kp) * K(k1)

    N = np.ceil(numerator / denominator)

    return int(N)


In [18]:
Rp = 1
Rs = 40

delta1 = np.sqrt(10**(Rp/10) - 1)
delta2 = np.sqrt(10**(Rs/10) - 1)

w_p = 1
w_s = 1.2

eps, k1, k1p, k, kp = compute_moduli(delta1, delta2, w_p, w_s)